# 重複しているコードをクラスにまとめる

* データベースに接続する部分を、共通して使うクラスとしてまとめる
    * データベースの変更や、パスワードの変更があったとしても1箇所の修正だけで済むようになる

## DB接続部分

In [45]:
%%bash
### 変数の設定
name=ConnectDb
classpath=".:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar"

### ソースの編集
cat <<- EOS > ${name}.java
import java.sql.Connection;
import java.sql.DriverManager;

class ${name} {
  static Connection getConnection() throws Exception{
    Connection con = null;
    
    //# 接続
    con = DriverManager.getConnection("jdbc:postgresql://10.136.3.189:5432/jsd_development", "jsd", "pass");
    return con;
  }
}
EOS

### コンパイル(デバッグ目的。本来はコンパイルしなくても他のコードをコンパイルすると自動でこのコードもコンパイルされる)
javac -encoding UTF-8 ${name}.java

## select

In [70]:
%%bash
### 変数の設定
name=PeopleSelect
classpath=".:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar"

### ソースの編集
cat <<- EOS > ${name}.java
import java.sql.Connection;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.Statement;
import java.util.ArrayList;
import java.util.List;

public class ${name} {
  public static void main(String[] args) throws Exception{
    Statement statement = null;
    ResultSet resultSet = null;
    Connection connection = null;
    
    try{
      //# 接続したオブジェクトをConnectDbから取得し、Statementオブジェクトを生成
      //# ここでConnection connectionとすろとfinallyでcannot find symbolのエラーになる
      //# https://stackoverflow.com/questions/15281640/cannot-find-symbol-when-trying-to-close-i-o-objects
      //# Connection connection = ConnectDb.getConnection();
      connection = ConnectDb.getConnection();
      statement = connection.createStatement();
      
      //# SQL文発行
      resultSet = statement.executeQuery("SELECT * FROM people");

      //# 値の取得
      List<String> fields = new ArrayList<String>();
      ResultSetMetaData rsmd = resultSet.getMetaData();
      for (int i = 1; i <= rsmd.getColumnCount(); i++) {
        fields.add(rsmd.getColumnName(i));
      }

      //# 結果の出力
      int rowCount = 0;
      while (resultSet.next()) {
        rowCount++;

        System.out.println("---------------------------------------------------");
        System.out.println("--- Rows:" + rowCount);
        System.out.println("---------------------------------------------------");

        //# 値は、「resultSet.getString(<フィールド名>)」で取得する。
        for (String field : fields) {
          System.out.println(field + ":" + resultSet.getString(field));
        }
      }
    } finally {
      //# 接続を切断する
      if (resultSet != null) {
        resultSet.close();
      }
      if (statement != null) {
        statement.close();
      }
      if (connection != null) {
        connection.close();
      }
    }
  }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

### 実行
java -classpath ${classpath} ${name}

---------------------------------------------------
--- Rows:1
---------------------------------------------------
id:1
name:Taro
age:21
address:Tokyo
---------------------------------------------------
--- Rows:2
---------------------------------------------------
id:2
name:Hanako
age:18
address:Hokkaido
---------------------------------------------------
--- Rows:3
---------------------------------------------------
id:10
name:Shiro
age:15
address:Ehime
---------------------------------------------------
--- Rows:4
---------------------------------------------------
id:11
name:Shiro
age:15
address:Ehime
---------------------------------------------------
--- Rows:5
---------------------------------------------------
id:12
name:Goro
age:29
address:Kochi
---------------------------------------------------
--- Rows:6
---------------------------------------------------
id:9
name:Ume
age:19
address:Wakayama


## insert

In [47]:
%%bash
### 変数の設定
name=PeopleInsert
classpath=".:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar"

### ソースの編集
cat <<- EOS > ${name}.java
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.Statement;
import java.util.ArrayList;
import java.util.List;

public class ${name} {
  public static void main(String[] args) throws Exception{
    Connection connection = null;
    Statement statement = null;
    ResultSet resultSet = null;
    
    if (args.length != 3) {
      System.out.println("usage: java -classpath .:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar PeopleInsert name age address");
      System.exit(1);
    }
    
    try{
      //# 接続
      connection = ConnectDb.getConnection();
      statement = connection.createStatement();
      
      //# SQL文発行
      String name = args[0];
      int age = Integer.parseInt(args[1]);
      String address = args[2];
      statement.executeUpdate("INSERT INTO people (name, age, address) VALUES ('" + name + "', " + age + ", '" + address + "')");

    } finally {
      //# 接続を切断する
      if (resultSet != null) {
        resultSet.close();
      }
      if (statement != null) {
        statement.close();
      }
      if (connection != null) {
        connection.close();
      }
    }
  }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

### 実行。ここで名前、年齢、住所を引数で与える
java -classpath ${classpath} ${name} Shiro 15 Ehime

## update

In [ ]:
%%bash
### 変数の設定
name=PeopleUpdate
classpath=".:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar"

### ソースの編集
cat <<- EOS > ${name}.java
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.Statement;
import java.util.ArrayList;
import java.util.List;

public class ${name} {
  public static void main(String[] args) throws Exception{
    Connection connection = null;
    Statement statement = null;
    ResultSet resultSet = null;
    
    if (args.length != 4) {
      System.out.println("usage: java -classpath .:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar PeopleUpdate name age address id");
      System.exit(1);
    }
    
    try{
      //# 接続
      connection = ConnectDb.getConnection();
      statement = connection.createStatement();
      
      //# SQL文発行
      String name = args[0];
      int age = Integer.parseInt(args[1]);
      String address = args[2];
      int id = Integer.parseInt(args[3]);
      statement.executeUpdate("UPDATE people SET name = '" + name + "', age = " + age + ", address = '" + address + "' WHERE id = " + id);
      //# DEBUG
      //# System.out.println("UPDATE people SET name = '" + name + "', age = " + age + ", address = '" + address + "' WHERE id = " + id);

    } finally {
      //# 接続を切断する
      if (resultSet != null) {
        resultSet.close();
      }
      if (statement != null) {
        statement.close();
      }
      if (connection != null) {
        connection.close();
      }
    }
  }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

### 実行。ここで名前、年齢、住所、更新対象のidを引数で与える
java -classpath ${classpath} ${name} Ume 19 Wakayama 9

## delete

In [69]:
%%bash
### 変数の設定
name=PeopleDelete
classpath=".:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar"

### ソースの編集
cat <<- EOS > ${name}.java
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.ResultSetMetaData;
import java.sql.Statement;
import java.util.ArrayList;
import java.util.List;

public class ${name} {
  public static void main(String[] args) throws Exception{
    Connection connection = null;
    Statement statement = null;
    ResultSet resultSet = null;
    
    if (args.length != 1) {
      System.out.println("usage: java -classpath .:/root/git_jupyter_notebook/Java/postgresql-42.1.1.jar PeopleDelete id");
      System.exit(1);
    }
    
    try{
      //# 接続
      connection = ConnectDb.getConnection();
      statement = connection.createStatement();
      
      //# SQL文発行
      int id = Integer.parseInt(args[0]);
      statement.executeUpdate("DELETE FROM people WHERE id = " + id);
      //# DEBUG
      //#System.out.println("DELETE FROM people WHERE id = " + id);

    } finally {
      //# 接続を切断する
      if (resultSet != null) {
        resultSet.close();
      }
      if (statement != null) {
        statement.close();
      }
      if (connection != null) {
        connection.close();
      }
    }
  }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

### 実行。ここで削除対象のidを引数で与える
java -classpath ${classpath} ${name} 8